In [1]:
### imports

# external modules
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib
from sklearn.preprocessing import StandardScaler

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import FlexiStruct
importlib.reload(FlexiStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-08-01 19:42:04.251651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

### Controls and Data Selection

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

In [3]:
### Evaluation Parameters

# Select the bias towards recall against precision, treated as a factor (so < 1 biases towards precision, 1 is equal importance, and > 1 biases towards recall)
wpBiasFactor = 20
fmBiasFactor = 2

In [4]:
### Defining bad runs
badruns = {'2017B':
                [
                    297048,
                    297282,
                    297283,
                    297284,
                    297287,
                    297288,
                    297289,
                    299316,
                    299317,
                    299318,
                    299324,
                    299326,
                    301086,
                    301086,
                    303948,
                    297047, #close but, true bad for all 8
                    297169, #true bad for all 8
                    297211, #Reconstructs well
                    299325, #Reconstructs well
                    297664, #true bad for all 8
                    299317, #true bad for all 8
                    297169, #true bad for all 8
                    297502
                ],
             '2017C':[
                  300781, # bad for tracking (pixels were excluded.
                  300079, # is bad for strips and then also for tracking
                  302029, # Poor detector elements for strips - Should be mildly anomalous, but technically good 
                  300576, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300574, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300282, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  301912, # Half bad for pixels (lost HV or readout card)  
                  301086, # Half bad for pixels (lost HV or readout card)  
                  300283, # Half bad for pixels (lost HV or readout card) 
                  300282, # Half bad for pixels (lost HV or readout card) 
                  300281, # Half bad for pixels (lost HV or readout card) 
                  300239, # Half bad for pixels (lost HV or readout card)
                  301394, # Marginal for pixels
                  301183, # Marginal for pixels
                  300398, # Marginal for pixels
                  300389, # Marginal for pixels
                  300365  # Marginal for pixels
             ],
             '2017E':[
                 304740, # Bad for pixels and tracking - holes in PXLayer 1
                 304776, # Bad for pixels and tracking - holes in PXLayer 1
                 304506, # Portcard problem for pixels
                 304507, # Portcard problem for pixels 
                 303989, # Bad for pixels, power supply died
                 303824  # Partly bad for strips due to a test
             ],
             '2017F':[
                 306422, # Partly bad for strips - 2 data readouts failed 
                 306423, # Partly bad for strips - 2 data readouts failed
                 306425, # Partly bad for strips - 2 data readouts failed
                 305440, # Partly bad for strips - 1 data readout failed
                 305441, # Partly bad for strips - 1 data readout failed
                 305249, # Bad for pixels - half of disk failed 
                 305250, # Bad for pixels - half of disk failed
                 305064, # Marginal for pixels - some readout failed
             ],
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                [
                317479,
                317480,
                317481,
                317482,
                319847
                ]}


In [5]:
### Select a reference run and get data
rundict = jsonu.loadjson('../jsons/CertHelperRefRuns.json')

# Select any run numbers to get a training set from that run's reference.
runNums = [303824, 306422]
refRuns = []
eras = []
years = []
dataDict = {}
badrunsls = {}
trainrunsls = {}
goodrunsls = {}
for runNum in runNums:
    runls = {}
    for run in rundict:
        if run['run_number'] == runNum:
            runls.update(run)
    if runls == {}:
        raise Exception('Run not found - ' + str(runNum))
    
    year = runls['dataset'][11:15]
    if year not in years: years.append(year)
    era = runls['dataset'][15]
    if era not in eras: eras.append(era)
    ref_run = runls['reference_run_number']
    
    # Don't need duplicates
    if ref_run in refRuns:
        continue
    refRuns.append(ref_run)
    
    # Get the runs associated with found reference
    outputRuns = {}
    outputBad = {}
    for run in rundict:
        tempRef = run['reference_run_number']
        if tempRef == ref_run:
            runls = {}
            runls[str(run['run_number'])] = [[-1]]
            if run['run_number'] in badruns[year+era]:
                if run['run_number'] == 303824:
                    continue
                print('Found bad run :' + str(run))
                outputBad.update(runls)
            else:
                outputRuns.update(runls)
    
    # Perform structuring for compatibility with autoencoders
    dataDict[year + era] = outputRuns
    badrunsls[year + era] = outputBad
    trainrunsls[year + era] = {}
    goodrunsls[year + era] = {}
    
    # Select training and testing set
    for i,run in enumerate(dataDict[year + era]):
        if i > 5 and i < 11:
            goodrunsls[year + era][str(run)] = [[-1]]
        else:
            trainrunsls[year + era][str(run)] = [[-1]]

if len(years) != 1: raise Exception('Year of length 0 or >1 unimplemented!')

Found bad run :{'run_number': 303989, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 304158, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017E_UL2019/DQM'}
Found bad run :{'run_number': 304740, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 304158, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017E_UL2019/DQM'}
Found bad run :{'run_number': 305250, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 306459, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017F_UL2019/DQM'}
Found bad run :{'run_number': 306422, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 306459, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017F_UL2019/DQM'}
Found bad run :{'run_number': 305249, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 306459, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017F_UL2019/DQM'}


In [6]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = {}
for era in eras:
    datadir[year + era] = '../data/' + year + era + '/'

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [['chargeInner_PXLayer_1', 'chargeInner_PXLayer_2', 'chargeInner_PXLayer_3', 'chargeInner_PXLayer_4'], ['charge_PXDisk_+1', 'charge_PXDisk_+2', 'charge_PXDisk_+3'], ['charge_PXDisk_-1', 'charge_PXDisk_-2', 'charge_PXDisk_-3'], ['Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'], ['Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__5', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__6'], ['Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__9'], ['Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9'], ['NumberOfRecHitsPerTrack_lumiFlag_GenTk']]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [7]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

In [8]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

In [9]:
### Define Training Mode Parameters - Combined Autoencoder
if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use averages of training set)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    # - this only works for a single era
    
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+era+'_'+histnames[0][0]+'.csv') ) )
    # Cherry picked really bad run
    run_application = 299316
    #run_application = 299317
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        
        # Selects the 5 previous runs for training
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)
        
elif training_mode == 'development':
    # train on a user-defined subset of runs
    
   # Select runs to be used in training from the user-defined list
    runsls_training = {}
    runsls_bad = {}
    runsls_good = {}
    for era in eras:
        runsls_training.update(trainrunsls[year + era])
        # Select bad runs to test on in the user-defined list
        runsls_bad.update(badrunsls[year + era])
        # Select good runs to test on in the user-defined list
        runsls_good.update(goodrunsls[year + era])
    
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

selected runs/lumisections for training: 
{'303819': [[-1]], '303999': [[-1]], '304119': [[-1]], '304120': [[-1]], '304197': [[-1]], '304505': [[-1]], '304449': [[-1]], '304452': [[-1]], '304508': [[-1]], '304625': [[-1]], '304655': [[-1]], '304737': [[-1]], '304778': [[-1]], '306459': [[-1]], '304196': [[-1]], '305310': [[-1]], '305040': [[-1]], '305043': [[-1]], '305185': [[-1]], '305204': [[-1]], '305234': [[-1]], '305376': [[-1]], '306042': [[-1]], '306051': [[-1]], '305406': [[-1]], '306122': [[-1]], '306134': [[-1]], '306137': [[-1]], '306154': [[-1]], '306170': [[-1]], '306417': [[-1]], '306432': [[-1]], '306456': [[-1]], '305516': [[-1]], '305586': [[-1]], '305588': [[-1]], '305590': [[-1]], '305809': [[-1]], '305832': [[-1]], '305840': [[-1]], '305898': [[-1]], '306029': [[-1]], '306037': [[-1]], '306095': [[-1]]}
selected runs/lumisections as good test set:
{'304198': [[-1]], '304199': [[-1]], '304209': [[-1]], '304333': [[-1]], '304446': [[-1]], '305247': [[-1]], '305313': [

### Data Import and Preprocessing

In [10]:
### Data Import

# Create a new HistStruct from the data
if readnew:
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = FlexiStruct.FlexiStruct()
    histstruct.reset_histlist(histnames)
    
    # Unpack histnames and add every histogram individually
    for era in eras:
        for histnamegroup in histnames:
            for histname in histnamegroup:
                print('Adding {}...'.format(histname + era))
                
                # Bring the histograms into memory from storage for later use
                filename = datadir[year+era] + 'DF' + year + era + '_' + histname + '.csv'
                df = dloader.get_dataframe_from_file( filename )
                
                # In case of local training, we can remove most of the histograms
                if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                    runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                    df = dfu.select_runsls( df, runsls_total )    
                
                df = dfu.rm_duplicates(df)
                # Store the data in the histstruct object managing this whole thing
                histstruct.add_dataframe( df, rebinningfactor = 1, standardbincount = 102 )
        print('Found {} histograms\n'.format(len(histstruct.runnbs)))

# Load a previously saved HistStruct
else:
    # Load histstruct from storage
    histstruct = SubHistStruct.SubHistStruct.load( 'histstruct_global_20220201.zip', verbose=False )
    nbadruns = len([name for name in list(histstruct.masks.keys()) if ('bad' in name and name!='bad')])
    
    print('loaded a histstruct with the following properties:')
    print(histstruct)
    # Count of bad runs, presumably for later use
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
print('Created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Adding chargeInner_PXLayer_1E...
Adding chargeInner_PXLayer_2E...
Adding chargeInner_PXLayer_3E...
Adding chargeInner_PXLayer_4E...
Adding charge_PXDisk_+1E...
Adding charge_PXDisk_+2E...
Adding charge_PXDisk_+3E...
Adding charge_PXDisk_-1E...
Adding charge_PXDisk_-2E...
Adding charge_PXDisk_-3E...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__1E...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__2E...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__3E...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__4E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__1E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__2E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__3E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__4E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__5E...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__6E...
Adding Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1E...
Adding Summary_ClusterStoNCorr__OnTrack__TID_

Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8F...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9F...
Adding NumberOfRecHitsPerTrack_lumiFlag_GenTkF...
Found 24978 histograms

Created a histstruct with the following properties:
- number of histogram types: 45
- number of lumisections: 24978


In [11]:
### Add Masks to Data

if readnew:
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat', entries_to_bins_ratio=0 )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=0 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        print(runsls_bad)
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    if save:
        histstruct.save('test.pk1')
print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

{'303989': [[-1]], '304740': [[-1]], '305250': [[-1]], '306422': [[-1]], '305249': [[-1]]}
Assigned masks: ['dcson', 'golden', 'highstat', 'lowstat', 'training', 'good', 'bad', 'bad0', 'bad1', 'bad2', 'bad3', 'bad4']


### Plotting and Data Validation

In [12]:
### Plotting the input data for analysis

if((training_mode=='local' or training_mode == 'development') and createPlots):

    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad0']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
elif( training_mode=='global' and createPlots):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                    )

### Concatamash Autoencoder

In [13]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            # Half the total bin count
            arch = 51 * len(histnamegroup)
            
            ## Model parameters
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(arch * 2, activation="tanh")(conc_layer)
            encoder = Dense(arch/2, activation='relu')(encoder)
            encoder = Dense(arch/8, activation='relu')(encoder)
            encoder = Dense(arch/16, activation='relu')(encoder)
            decoder = Dense(arch/8, activation="relu")(encoder)
            decoder = Dense(arch/2, activation='relu')(encoder)
            decoder = Dense(arch * 2, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [14]:
(histslist, vallist, autoencoders, train_normhist) = define_concatamash_autoencoder(histstruct)

2022-08-01 19:46:15.248266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_fo

In [15]:
### Trains a combined autoencoder for every merge set
def train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders):
    
    autoencodersTrain = []
    for i in range(len(histslist)):
        
        print('Now training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 500
        batch_size = 10000
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                                epochs=nb_epoch,
                                batch_size=batch_size,
                                shuffle=True,
                                validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                                verbose=1,
                                callbacks= [earlystop],    
                                )
        tf.keras.utils.plot_model(
                    autoencoder,
                    to_file="../Output Data/models/model1D{}.png".format(i),
                    show_shapes=True,
                    show_dtype=False,
                    show_layer_names=False,
                    rankdir="TB")
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier) 
        autoencodersTrain.append(classifier)
        K.clear_session()
        del(autoencoder, classifier)
    return autoencodersTrain

In [16]:
### Functionality doesn't seem to work at this time, so this function is empty
def load_concatamash_autoencoder():
    pass

In [17]:
start = time.perf_counter()
if trainnew: train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders)
else: load_concatamash_autoencoder()
stop = time.perf_counter()
print(stop - start)

Now training model 1/10
Epoch 1/500
2/2 [==============================] - 3s 563ms/step - loss: 0.0014 - dense_7_loss: 3.0165e-04 - dense_8_loss: 3.7979e-04 - dense_9_loss: 3.6540e-04 - dense_10_loss: 3.4805e-04 - val_loss: 0.0010 - val_dense_7_loss: 2.1519e-04 - val_dense_8_loss: 2.8600e-04 - val_dense_9_loss: 2.5732e-04 - val_dense_10_loss: 2.4411e-04
Epoch 2/500
2/2 [==============================] - 0s 146ms/step - loss: 9.7578e-04 - dense_7_loss: 2.0877e-04 - dense_8_loss: 2.7950e-04 - dense_9_loss: 2.5095e-04 - dense_10_loss: 2.3657e-04 - val_loss: 6.2910e-04 - val_dense_7_loss: 1.2820e-04 - val_dense_8_loss: 1.9225e-04 - val_dense_9_loss: 1.6585e-04 - val_dense_10_loss: 1.4281e-04
Epoch 3/500
2/2 [==============================] - 0s 200ms/step - loss: 6.0566e-04 - dense_7_loss: 1.2295e-04 - dense_8_loss: 1.8583e-04 - dense_9_loss: 1.5976e-04 - dense_10_loss: 1.3711e-04 - val_loss: 3.1453e-04 - val_dense_7_loss: 6.0925e-05 - val_dense_8_loss: 1.0348e-04 - val_dense_9_loss: 8.29

Epoch 25/500
2/2 [==============================] - 0s 129ms/step - loss: 3.4622e-05 - dense_7_loss: 8.3467e-06 - dense_8_loss: 1.2157e-05 - dense_9_loss: 9.5011e-06 - dense_10_loss: 4.6175e-06 - val_loss: 3.3395e-05 - val_dense_7_loss: 7.9114e-06 - val_dense_8_loss: 1.1841e-05 - val_dense_9_loss: 9.1593e-06 - val_dense_10_loss: 4.4827e-06
Epoch 26/500
2/2 [==============================] - 0s 128ms/step - loss: 3.3857e-05 - dense_7_loss: 8.0969e-06 - dense_8_loss: 1.1987e-05 - dense_9_loss: 9.3078e-06 - dense_10_loss: 4.4656e-06 - val_loss: 3.2711e-05 - val_dense_7_loss: 7.6722e-06 - val_dense_8_loss: 1.1688e-05 - val_dense_9_loss: 8.9883e-06 - val_dense_10_loss: 4.3626e-06
Epoch 27/500
2/2 [==============================] - 0s 132ms/step - loss: 3.3171e-05 - dense_7_loss: 7.8643e-06 - dense_8_loss: 1.1825e-05 - dense_9_loss: 9.1328e-06 - dense_10_loss: 4.3489e-06 - val_loss: 3.2034e-05 - val_dense_7_loss: 7.5319e-06 - val_dense_8_loss: 1.1427e-05 - val_dense_9_loss: 8.7875e-06 - val_

Epoch 49/500
2/2 [==============================] - 0s 130ms/step - loss: 8.6972e-06 - dense_7_loss: 3.0400e-06 - dense_8_loss: 2.2594e-06 - dense_9_loss: 1.9838e-06 - dense_10_loss: 1.4140e-06 - val_loss: 8.5301e-06 - val_dense_7_loss: 2.8240e-06 - val_dense_8_loss: 2.3158e-06 - val_dense_9_loss: 1.9317e-06 - val_dense_10_loss: 1.4586e-06
Epoch 50/500
2/2 [==============================] - 0s 141ms/step - loss: 8.3576e-06 - dense_7_loss: 2.8750e-06 - dense_8_loss: 2.2053e-06 - dense_9_loss: 1.8978e-06 - dense_10_loss: 1.3794e-06 - val_loss: 6.5374e-06 - val_dense_7_loss: 2.5728e-06 - val_dense_8_loss: 1.4965e-06 - val_dense_9_loss: 1.3267e-06 - val_dense_10_loss: 1.1415e-06
Epoch 51/500
2/2 [==============================] - 0s 123ms/step - loss: 6.6059e-06 - dense_7_loss: 2.5982e-06 - dense_8_loss: 1.5142e-06 - dense_9_loss: 1.3780e-06 - dense_10_loss: 1.1155e-06 - val_loss: 6.9082e-06 - val_dense_7_loss: 2.7080e-06 - val_dense_8_loss: 1.6037e-06 - val_dense_9_loss: 1.4235e-06 - val_

Epoch 73/500
2/2 [==============================] - 0s 129ms/step - loss: 4.5105e-06 - dense_7_loss: 2.0205e-06 - dense_8_loss: 8.7549e-07 - dense_9_loss: 8.1815e-07 - dense_10_loss: 7.9634e-07 - val_loss: 4.4979e-06 - val_dense_7_loss: 1.9869e-06 - val_dense_8_loss: 8.8392e-07 - val_dense_9_loss: 7.8915e-07 - val_dense_10_loss: 8.3795e-07
Epoch 74/500
2/2 [==============================] - 0s 129ms/step - loss: 4.4768e-06 - dense_7_loss: 2.0171e-06 - dense_8_loss: 8.6111e-07 - dense_9_loss: 8.0800e-07 - dense_10_loss: 7.9058e-07 - val_loss: 4.4780e-06 - val_dense_7_loss: 1.9922e-06 - val_dense_8_loss: 8.7194e-07 - val_dense_9_loss: 7.8164e-07 - val_dense_10_loss: 8.3218e-07
Epoch 75/500
2/2 [==============================] - 0s 151ms/step - loss: 4.4589e-06 - dense_7_loss: 2.0186e-06 - dense_8_loss: 8.5180e-07 - dense_9_loss: 8.0207e-07 - dense_10_loss: 7.8640e-07 - val_loss: 4.4903e-06 - val_dense_7_loss: 2.0025e-06 - val_dense_8_loss: 8.7346e-07 - val_dense_9_loss: 7.8353e-07 - val_

Epoch 11/500
2/2 [==============================] - 0s 96ms/step - loss: 4.1272e-05 - dense_18_loss: 1.3736e-05 - dense_19_loss: 1.3742e-05 - dense_20_loss: 1.3794e-05 - val_loss: 3.2000e-05 - val_dense_18_loss: 1.0329e-05 - val_dense_19_loss: 1.1659e-05 - val_dense_20_loss: 1.0012e-05
Epoch 12/500
2/2 [==============================] - 0s 90ms/step - loss: 3.1781e-05 - dense_18_loss: 1.0271e-05 - dense_19_loss: 1.1677e-05 - dense_20_loss: 9.8330e-06 - val_loss: 2.7524e-05 - val_dense_18_loss: 9.5301e-06 - val_dense_19_loss: 1.0004e-05 - val_dense_20_loss: 7.9901e-06
Epoch 13/500
2/2 [==============================] - 0s 98ms/step - loss: 2.7535e-05 - dense_18_loss: 9.5763e-06 - dense_19_loss: 1.0039e-05 - dense_20_loss: 7.9194e-06 - val_loss: 2.5751e-05 - val_dense_18_loss: 9.7327e-06 - val_dense_19_loss: 8.5735e-06 - val_dense_20_loss: 7.4448e-06
Epoch 14/500
2/2 [==============================] - 0s 88ms/step - loss: 2.5867e-05 - dense_18_loss: 9.7948e-06 - dense_19_loss: 8.6313e-06

...

Epoch 66/500
2/2 [==============================] - 2s 487ms/step - loss: 1.7931e-05 - dense_82_loss: 2.8808e-06 - dense_83_loss: 1.9990e-06 - dense_84_loss: 1.9073e-06 - dense_85_loss: 2.3746e-06 - dense_86_loss: 1.8515e-06 - dense_87_loss: 1.9787e-06 - dense_88_loss: 1.7054e-06 - dense_89_loss: 1.9049e-06 - dense_90_loss: 1.3293e-06 - val_loss: 1.5979e-05 - val_dense_82_loss: 2.1332e-06 - val_dense_83_loss: 1.8085e-06 - val_dense_84_loss: 1.8764e-06 - val_dense_85_loss: 1.6127e-06 - val_dense_86_loss: 1.6053e-06 - val_dense_87_loss: 1.7994e-06 - val_dense_88_loss: 1.6970e-06 - val_dense_89_loss: 1.4865e-06 - val_dense_90_loss: 1.9604e-06
Epoch 67/500
2/2 [==============================] - 2s 575ms/step - loss: 1.7645e-05 - dense_82_loss: 2.8335e-06 - dense_83_loss: 1.9526e-06 - dense_84_loss: 1.8696e-06 - dense_85_loss: 2.3460e-06 - dense_86_loss: 1.8261e-06 - dense_87_loss: 1.9480e-06 - dense_88_loss: 1.6776e-06 - dense_89_loss: 1.8859e-06 - dense_90_loss: 1.3059e-06 - val_loss: 1.5

Epoch 79/500
2/2 [==============================] - 1s 486ms/step - loss: 1.5099e-05 - dense_82_loss: 2.3900e-06 - dense_83_loss: 1.5938e-06 - dense_84_loss: 1.5321e-06 - dense_85_loss: 2.0930e-06 - dense_86_loss: 1.5902e-06 - dense_87_loss: 1.6501e-06 - dense_88_loss: 1.4386e-06 - dense_89_loss: 1.6830e-06 - dense_90_loss: 1.1283e-06 - val_loss: 1.3303e-05 - val_dense_82_loss: 1.6625e-06 - val_dense_83_loss: 1.4353e-06 - val_dense_84_loss: 1.5234e-06 - val_dense_85_loss: 1.3471e-06 - val_dense_86_loss: 1.3529e-06 - val_dense_87_loss: 1.4899e-06 - val_dense_88_loss: 1.4496e-06 - val_dense_89_loss: 1.2728e-06 - val_dense_90_loss: 1.7695e-06
Epoch 80/500
2/2 [==============================] - 2s 486ms/step - loss: 1.4913e-05 - dense_82_loss: 2.3569e-06 - dense_83_loss: 1.5706e-06 - dense_84_loss: 1.5086e-06 - dense_85_loss: 2.0745e-06 - dense_86_loss: 1.5726e-06 - dense_87_loss: 1.6277e-06 - dense_88_loss: 1.4200e-06 - dense_89_loss: 1.6671e-06 - dense_90_loss: 1.1147e-06 - val_loss: 1.3

Epoch 92/500
2/2 [==============================] - 1s 473ms/step - loss: 1.3355e-05 - dense_82_loss: 2.0752e-06 - dense_83_loss: 1.3839e-06 - dense_84_loss: 1.3157e-06 - dense_85_loss: 1.9147e-06 - dense_86_loss: 1.4267e-06 - dense_87_loss: 1.4449e-06 - dense_88_loss: 1.2575e-06 - dense_89_loss: 1.5394e-06 - dense_90_loss: 9.9648e-07 - val_loss: 1.1687e-05 - val_dense_82_loss: 1.3680e-06 - val_dense_83_loss: 1.2432e-06 - val_dense_84_loss: 1.3243e-06 - val_dense_85_loss: 1.1797e-06 - val_dense_86_loss: 1.1979e-06 - val_dense_87_loss: 1.3041e-06 - val_dense_88_loss: 1.2836e-06 - val_dense_89_loss: 1.1417e-06 - val_dense_90_loss: 1.6446e-06
Epoch 93/500
2/2 [==============================] - 1s 458ms/step - loss: 1.3265e-05 - dense_82_loss: 2.0599e-06 - dense_83_loss: 1.3734e-06 - dense_84_loss: 1.3043e-06 - dense_85_loss: 1.9054e-06 - dense_86_loss: 1.4183e-06 - dense_87_loss: 1.4345e-06 - dense_88_loss: 1.2485e-06 - dense_89_loss: 1.5315e-06 - dense_90_loss: 9.8960e-07 - val_loss: 1.1

Epoch 105/500
2/2 [==============================] - 2s 681ms/step - loss: 1.2586e-05 - dense_82_loss: 1.9481e-06 - dense_83_loss: 1.2867e-06 - dense_84_loss: 1.2130e-06 - dense_85_loss: 1.8307e-06 - dense_86_loss: 1.3528e-06 - dense_87_loss: 1.3620e-06 - dense_88_loss: 1.1827e-06 - dense_89_loss: 1.4736e-06 - dense_90_loss: 9.3633e-07 - val_loss: 1.1000e-05 - val_dense_82_loss: 1.2534e-06 - val_dense_83_loss: 1.1559e-06 - val_dense_84_loss: 1.2330e-06 - val_dense_85_loss: 1.1022e-06 - val_dense_86_loss: 1.1283e-06 - val_dense_87_loss: 1.2342e-06 - val_dense_88_loss: 1.2177e-06 - val_dense_89_loss: 1.0852e-06 - val_dense_90_loss: 1.5903e-06
Epoch 106/500
2/2 [==============================] - 2s 468ms/step - loss: 1.2554e-05 - dense_82_loss: 1.9429e-06 - dense_83_loss: 1.2820e-06 - dense_84_loss: 1.2084e-06 - dense_85_loss: 1.8269e-06 - dense_86_loss: 1.3495e-06 - dense_87_loss: 1.3587e-06 - dense_88_loss: 1.1798e-06 - dense_89_loss: 1.4711e-06 - dense_90_loss: 9.3422e-07 - val_loss: 1

KeyboardInterrupt: 

In [ ]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        print('evaluating model for '+histgroup[0])
        print(histstruct.evaluate_classifier(histgroup)[0].shape)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    print('Found mse array for training set of following shape: {}'.format(mse_train.shape))
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            print(run)
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    print('Found mse array for good set of following shape: {}'.format(mse_good.shape))
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        print('Found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))
        
    return [mse_train, mse_good, mse_bad]

In [ ]:
(mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)

In [ ]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct, mse_train):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    #for dims in dimslist:
    #    thismse = mse_train[:,dims]
    #    if training_mode=='global': 
    #        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
    #        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
    #        #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
    #        #                                                    'up')
    #    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    #    #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
    #    #                onlycontour=False, xlims=30, ylims=30, 
    #    #                onlypositive=True, transparency=0.5,
    #    #                xaxtitle=histstruct.histnames[dims[0]], 
    #    #                yaxtitle=histstruct.histnames[dims[1]],
    #    #                title='density fit of lumisection MSE')
    #    ##plt.close('all') # release plot memory
    #    fitfunclist.append(fitfunc)
    # 
    #    
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: 
        fitfunc = GaussianKdeFitter.GaussianKdeFitter()
        fitfunc.fit(mse_train,bw_method=bw_method)
    
    return fitfunc

In [ ]:
fitfunc = fit_mse_distribution(histstruct, mse_train)

In [ ]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    print('--- good lumesections ---')
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    print('length of log prob array: '+str(len(logprob_good)))
    print('minimum of log prob: '+str(np.min(logprob_good)))
    #print(sorted(logprob_good))
    
    print('--- bad lumisections ---')
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]

    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    print('length of log prob array: '+str(len(logprob_bad)))
    print('maximum of log prob: '+str(np.max(logprob_bad)))
    #print(sorted(logprob_good))
    #print(sorted(logprob_bad))
    #print(logprob_bad)
    
    sep = np.min(logprob_good) - np.max(logprob_bad)
    print('Separability: ' + str(sep))
    
    return [logprob_good, logprob_bad, logprob_bad_parts]

In [ ]:
(logprob_good, logprob_bad, logprob_bad_parts) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)

In [ ]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, -1))
    goodMax = max(np.where(logprob_good != np.inf, logprob_good, 10001))
    
    logprob_good = np.where(logprob_good != np.inf, logprob_good, goodMax)
    logprob_bad = np.where(logprob_bad != -np.inf, logprob_bad, badMin)
    
    # These only take effect if a histogram is grossly misclassified
    logprob_good[logprob_good == -np.inf] = badMin
    logprob_bad[logprob_bad == np.inf] = goodMax
    
    labels = np.loadtxt('./Labels.csv', delimiter = ',')
    labels = np.concatenate(tuple([labels_good, labels_bad]))
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    avSep = np.mean(logprob_good) - np.mean(logprob_bad)
    
    print('Average Separation: ' + str(avSep))
    
    goodPos = np.mean(logprob_good)
    badPos = np.mean(logprob_bad)
    
    # Setting a threshold, below this working point defines anomalous data
    # Average is biased towards better recall per user specifications
    logprob_threshold = (1/(wpBiasFactor + 1)) * (wpBiasFactor*np.mean(logprob_good) + np.mean(logprob_bad))
    # Or set manual
    #logprob_threshold = 100
    
    (fig, ax) = pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                       bcklabel='good', bckcolor='g', 
                       nbins=200, normalize=True,
                       xaxtitle='negative logarithmic probability',
                       yaxtitle='number of lumisections (normalized)', doshow=False)
    
    plt.axvline(x=-logprob_threshold, color='b', label='WP')
    plt.show()
      
    # Plot ROC curve for analysis
    auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)
    
    mis = np.where((labels==1) & (scores<-logprob_threshold),1,0).astype(bool)
    labels2 = np.where(mis, 0, labels)
    mis = np.where((labels2==0) & (scores>-logprob_threshold),1,0).astype(bool)
    labels2 = np.where(mis, 1, labels2)
    np.savetxt('./Labels.csv', labels2, delimiter=',')
    
    (_, _, _, tp, fp, tn, fn) = aeu.get_confusion_matrix(scores,labels,-logprob_threshold)
    print('Selected logprob threshold of ' + str(logprob_threshold))
    
    # Get metrics for analysis
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (1 + fmBiasFactor * fmBiasFactor) * ((precision * recall) / ((fmBiasFactor * fmBiasFactor * precision) + recall)) 
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-Measure: ' + str(f_measure))
    
    return logprob_threshold, goodPos, badPos

In [ ]:
logprob_threshold, goodPos, badPos = evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor)

In [ ]:
def evaluate_autoencoders_individual(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor, logprob_bad_parts, goodPos, badPos):
    for i,logprob_bad in enumerate(logprob_bad_parts):
        
        labels_good = np.zeros(len(logprob_good)) # background: label = 0
        labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
        
        badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, -1))
        goodMax = max(np.where(logprob_good != np.inf, logprob_good, 10001))
        
        logprob_good = np.where(logprob_good != np.inf, logprob_good, goodMax)
        logprob_bad = np.where(logprob_bad != -np.inf, logprob_bad, badMin)
        
        # These only take effect if a histogram is grossly misclassified
        logprob_good[logprob_good == -np.inf] = badMin
        logprob_bad[logprob_bad == np.inf] = goodMax
        
        labels = labels_bad
        scores = -logprob_bad
        scores = aeu.clip_scores( scores )
    
        print('Showing Data from Bad Run ' + str(i))
        (fig, ax) = pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='m', 
                           bcklabel='good', bckcolor='k', 
                           nbins=200, normalize=True,
                           xaxtitle='negative logarithmic probability',
                           yaxtitle='number of lumisections (normalized)', doshow=False)
        
        plt.axvline(x=-logprob_threshold, color='b', label='WP', linestyle='-')
        plt.axvline(x=-goodPos, color='g', label='Good Avg', linestyle='-.')
        plt.axvline(x=-badPos, color='r', label='Bad Avg', linestyle='--')
        plt.show()

In [ ]:
evaluate_autoencoders_individual(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor, logprob_bad_parts, goodPos, badPos)

In [ ]:
model = keras.models.load_model('../SavedModels/Permutations/Job1/AE0/')